# Los Angeles Real Estate Price Prediction

## Part 2: Explore Spatial Relationships

Real estate prices are something that are tied not only to the attributes of the property, such as the lot size, house size, and other variables, but also the conditions around them that exist in space. 
This could be demographics, nearby sales, neighborhoods, proximity to business districts, etc.

To truly understand and predict prices we need to look at the spatial variables that make up the conditions around a property. In this notebook we will explore that further using real estate sales data from 2017 to 2019 in Los Angeles County.

This data has been imported into CARTO, a spatial data science platform and PostGIS database, and will be brought into the notebook using CARTOFrames, a library that will allow us to interface with data in our CARTO account and create map visualizations. 

The data consists of two datasets, the first being historical sales data from the Los Angeles County Office of the Assessor:    

- [Property Assessment Information System Map](http://maps.assessor.lacounty.gov/GVH_2_2/Index.html?configBase=http://maps.assessor.lacounty.gov/Geocortex/Essentials/REST/sites/PAIS/viewers/PAIS_hv/virtualdirectory/Resources/Config/Default)
- [Sales Parcels](http://assessor.gis.lacounty.gov/assessor/rest/services/PAIS/pais_sales_parcels/MapServer/0)
- [All Parcels](http://assessor.gis.lacounty.gov/assessor/rest/services/PAIS/pais_parcels/MapServer/0)

And parcel information from the LA County Open Data Portal:

- [2018 Property Parcels](https://data.lacounty.gov/Parcel-/Assessor-Parcels-Data-2018/mk7y-hq5p)

Our goals for this project are:


1. To import and clean the real estate data 
2. Exploratory (spatial) data analysis
3. Limit our analysis to one category of property and explore relevant features
4. Feature engineering and tests with various machine learning models
5. Add spatial features to the data and explore spatial relationships
6. Test the model with spatial features to see the impact
7. Evaluate our model and deploy it for production usage


In [ ]:
import pandas as pd
import cartoframes
import seaborn as sns
import numpy as np
import libpysal
from cartoframes import Credentials
from cartoframes.contrib import vector

import matplotlib.pyplot as plt
%matplotlib inline

from IPython.core.pylabtools import figsize

pd.set_option('display.max_columns', 500)
sns.set_style("white")
sns.set_style("ticks")
sns.despine()


USERNAME = 'mforrest-isolines'  
APIKEY = '07ab3fb439d92c5f06cfec08bb3417d209c646d8'
creds = Credentials(username=USERNAME, key=APIKEY)

cc = cartoframes.CartoContext(creds=creds)

In [ ]:
sfr = cc.read('la_singlefamilyhomes')

In [ ]:
sfr.head(2)

In [ ]:
sfr.columns.values

In [ ]:
f = sfr[['bathrooms', 'bedrooms', 'yearbuilt', 'improvementvalue', 'landvalue',
         'saleprice', 'size', 'sqftmain']]

f['yearbuilt'] = f['yearbuilt'].astype(int)

f['saleprice'] = f['saleprice'].apply(lambda x:np.log(x))
f['improvementvalue'] = f['improvementvalue'].apply(lambda x:np.log(x).astype(int))
# f = f[(f['improvementvalue'] != float('inf')) and (f['improvementvalue'] != float('-inf'))]

f['landvalue'] = f['landvalue'].apply(lambda x:np.log(x).astype(int))
# f = f[(f['landvalue'] != float('inf')) and (f['landvalue'] != float('-inf'))]
f = f[(f['yearbuilt'] > 0) & (f['bedrooms'] < 20) & (f['bathrooms'] < 20) & (f['improvementvalue'] > 0)]

# f.reset_index(drop=True, inplace=True)

pp = sns.pairplot(f)

# Residential Area and Sale Price

In [ ]:
sns.set_style("white")
sns.set_style("ticks")

d = f[f['bedrooms'] < 20]

g = sns.JointGrid(x="saleprice", y="bedrooms", data=d)
g = g.plot(sns.regplot, sns.distplot)

In [ ]:
sns.set_style("white")
sns.set_style("ticks")

f['size'] = f['size'].apply(lambda x:np.log(x))

g = sns.JointGrid(x="saleprice", y="size", data=f)
g = g.plot(sns.regplot, sns.distplot)

In [ ]:
sns.set_style("white")
sns.set_style("ticks")

f['landvalue'] = f['landvalue'].apply(lambda x:np.log(x))

g = sns.JointGrid(x="saleprice", y="landvalue", data=f)
g = g.plot(sns.regplot, sns.distplot)

# Remove additional outliers

As we saw above, we want to pull out any additional outlier data.

In [ ]:
d = pd.Series(np.log(sfr.saleprice))

ax = sns.distplot(d, bins=20, kde=True, rug=True, color="#0A157F", axlabel='Sales Price')

# Remove extreme outliers

Remove the most extreme outliers from the data using this function:

In [ ]:
first_quartile = sfr['saleprice'].describe()['25%']
third_quartile = sfr['saleprice'].describe()['75%']

# Interquartile range
iqr = third_quartile - first_quartile

# Remove outliers
sfr_c = sfr[(sfr['saleprice'] > (first_quartile - 3 * iqr)) &
            (sfr['saleprice'] < (third_quartile + 3 * iqr))]

In [ ]:
d = pd.Series(np.log(sfr_c.saleprice))
d = sfr_c.saleprice

ax = sns.distplot(d, bins=20, kde=False, rug=True, color="#0A157F", axlabel='Sales Price')

In [ ]:
sfr_c.sort_values(by=['saleprice'], ascending=False).head(10)

# Spatial Data Exploration and Variables

Before we run our prediction model, let's look at the spatial relationships between the different buildings across the city. 

We will use [**PySAL**](https://pysal.org/index.html) or Python Spatial Analysis Library to perform the spatial data exploration. We will identify significant clusters of high home sales using the [`esda`](https://esda.readthedocs.io/en/latest/) module from PySAL.

In [ ]:
cc.write(sfr_c, 'la_eval_clean', overwrite=True)

In [ ]:
sfr_pysal = cc.read('la_eval_clean', decode_geom=True)
sfr_ps = sfr_pysal.sort_values(ascending=False, by='formatted_saledate')
sfr_ps.head(2)

In [ ]:
sfr_ps.info()

# Drop duplicate geometries

We need to drop duplicate geometries from the dataset for the Moran's I evaluation since that will cause an error with PySAL.

In [ ]:
sfr_ps.drop_duplicates(subset = "plot_id", inplace = True) 
sfr_ps.info()

# Create spatial weights

First we need to evaluate the spatial relationships between all the different buildings. Since these geometries do not touch, we want to use the KNN weights from PySAL:

https://libpysal.readthedocs.io/en/latest/generated/libpysal.weights.KNN.html#libpysal.weights.KNN

In [ ]:
W = libpysal.weights.KNN.from_dataframe(sfr_ps, k=10)
W.transform = 'r'

# Moran's I Local

To identify the significant clusters, we will use the Moran's I Local analysis from PySAL to identify clusters of high sale prices. Spatial autocorrelation as described by the PySAL examples is:

*The concept of spatial autocorrelation relates to the combination of two types of similarity: spatial similarity and attribute similarity. Although there are many different measures of spatial autocorrelation, they all combine these two types of similarity into a summary measure.*

http://darribas.org/gds_scipy16/ipynb_md/04_esda.html
https://nbviewer.jupyter.org/github/pysal/esda/blob/master/notebooks/Spatial%20Autocorrelation%20for%20Areal%20Unit%20Data.ipynb

In [ ]:
import esda
moran = esda.Moran_Local(sfr_ps.saleprice, W, transformation = "r")

# Moran's Quads

We will use these values to create human readable clusters from the analysis - from the PySAL docs:

**q : array**

(if permutations>0) values indicate quandrant location 1 HH, 2 LH, 3 LL, 4 HL

In [ ]:
moran.q[10:100]

# Similarity

From PySAL Docs:

**p_sim : array**

(if permutations>0) p-values based on permutations (one-sided) null: spatial randomness alternative: the observed Ii is further away or extreme from the median of simulated values. It is either extremely high or extremely low in the distribution of simulated Is.

In [ ]:
moran.p_sim

From PySAL Docs:

**p_z_sim : array**

(if permutations>0) p-values based on standard normal approximation from permutations (one-sided) for two-sided tests, these values should be multiplied by 2

In [ ]:
moran.p_z_sim

In [ ]:
lag = libpysal.weights.lag_spatial(W, sfr_ps.saleprice)
data = sfr_ps.saleprice

In [ ]:
sig = 1 * (moran.p_sim < 0.05)
HH = 1 * (sig * moran.q==1)
LL = 3 * (sig * moran.q==3)
LH = 2 * (sig * moran.q==2)
HL = 4 * (sig * moran.q==4)
spots = HH + LL + LH + HL
spots

In [ ]:
spot_labels = [ '0 Non-Significant', 'HH - Hot Spot', 'LH - Donut', 'LL - Cold Spot', 'HL - Diamond']
labels = [spot_labels[i] for i in spots]

In [ ]:
moran_to_carto = sfr_ps.assign(cl=labels, p_sim = moran.p_sim, p_z_sim = moran.p_z_sim)
moran_to_carto.head(2)

In [ ]:
cc.write(moran_to_carto, 'manhattan_moran', overwrite=True)

In [ ]:

buckets='buckets($cl,["HH - Hot Spot","HL - Diamond","LH - Donut","LL - Cold Spot"])';
colorRamp='ramp('+buckets+',[#cf597e, #e88471, #39b185, #009392])';
strokeRamp='ramp('+buckets+',[#B54E6F, #CF7765, #309671, #007A7A],#636363)';
opacityStart='opacity('+colorRamp+',1)';
opacityEnd='opacity('+colorRamp+',0.6)';

vector.vmap(
    [vector.Layer(
        'manhattan_moran',
        color='ramp(zoomrange([0,16]),['+opacityStart+','+opacityEnd+'])',
        strokeWidth='ramp(zoomrange([12,14]),[0,0.7])',
        strokeColor=strokeRamp,
        interactivity={
            'cols': ['cl','formatted_saleprice','formatted_size','generalusetype'],
            'event': 'hover'
        }
    ),],
    context=cc,
    basemap=vector.BaseMaps.voyager
)

# HH - Hot Spots

We can see that there are clusters of high sale price properties in the Upper East Side and the West Village

In [ ]:
qHH = '''
      SELECT * FROM manhattan_moran
      WHERE cl = 'HH - Hot Spot'
      '''

vector.vmap(
    [vector.QueryLayer(
        qHH,
        color='ramp(zoomrange([0,16]),[opacity(#cf597e,1),opacity(#cf597e,0.6)])',
        strokeWidth='ramp(zoomrange([12,14]),[0,0.7])',
        strokeColor='#B54E6F',
        interactivity={
            'cols': ['formatted_saleprice','formatted_size','generalusetype'],
            'header': ['<h2>HH - Hot Spot<h2>', ],
            'event': 'hover'
        }
    ),
    ],
    context=cc,
    basemap=vector.BaseMaps.voyager
)

# LL - Cold Spots

Upper Manhattan contains large clusters of low sale price properties

In [ ]:
qLL = '''
      SELECT * FROM manhattan_moran
      WHERE cl = 'LL - Cold Spot'
      '''

vector.vmap(
    [vector.QueryLayer(
        qLL,
        color='ramp(zoomrange([0,16]),[opacity(#009392,1),opacity(#009392,0.6)])',
        strokeWidth='ramp(zoomrange([12,14]),[0,0.7])',
        strokeColor='#007A7A',
        interactivity={
            'cols': ['formatted_saleprice','formatted_size','generalusetype'],
            'header': ['<h2>LL - Cold Spot<h2>', ],
            'event': 'hover'
        }
    ),],
    context=cc,
    basemap=vector.BaseMaps.voyager
)

# HL - Diamond

Properties of high sale price near low sale price properties are also concentrated in LA

In [ ]:
qHL = '''
      SELECT * FROM manhattan_moran
      WHERE cl = 'HL - Diamond'
      '''

vector.vmap(
    [vector.QueryLayer(
        qHL,
        color='ramp(zoomrange([0,16]),[opacity(#e88471,1),opacity(#e88471,0.6)])',
        strokeWidth='ramp(zoomrange([12,14]),[0,0.7])',
        strokeColor='#CF7765',
        interactivity={
            'cols': ['formatted_saleprice','formatted_size','generalusetype'],
            'header': ['<h2>HL - Diamond<h2>', ],
            'event': 'hover'
        }
    ),],
    context=cc,
    basemap=vector.BaseMaps.voyager
)

# LH - Donut

Low sale price properties near high sale price properties are scattered across the city

In [ ]:
qLH = '''
      SELECT * FROM manhattan_moran
      WHERE cl = 'LH - Donut'
      '''

colorRamp='ramp('+buckets+',[#cf597e, #e88471, #39b185, #009392])';
strokeRamp='ramp('+buckets+',[#B54E6F, #CF7765, #309671, #007A7A],#636363)';


vector.vmap(
    [vector.QueryLayer(
        qLH,
        color='ramp(zoomrange([0,16]),[opacity(#39b185,1),opacity(#39b185,0.6)])',
        strokeWidth='ramp(zoomrange([12,14]),[0,0.7])',
        strokeColor='#309671',
        interactivity={
            'cols': ['formatted_saleprice','formatted_size','generalusetype'],
            'header': ['<h2>LH - Donut<h2>', ],
            'event': 'hover'
        }
    ),],
    context=cc,
    basemap=vector.BaseMaps.voyager
)